# Tech Intro

## Main tables

In [19]:
# Real Estate's objects
real_estate_list: list = [
    'ag_object',
    'ag_parse_ad',
    'ag_lead_object',
    'ag_presentation_object',
    'ag_object_platform',
    'ag_posting_ad_realty',
    'native_realty_filial',
    'foreign_realty_filial',
    'spr_metro',
    'spr_metro_connection',
    'ag_object_parking']
# Lead's objects
lead_list: list = [
    'leads',
    'lead_utm',
    'client_wishes',
    'leads_email_order',
    'request_from_site',
    'lead_finance',
    'leads_source_spr',
    'leads_calls_connection',
    'leads_pipeline',
    'leads_not_implemented_reason',
    'leads_status',
    'ag_pipeline',
    'ag_pipeline_status',
    'leads_meta',
    'leads_meta_fields',
    'leads_meta_group',
    'leads_contacts_connect',
    'leads_contact',
    'leads_contact_meta',
    'leads_contact_meta_fields']
# Deal's objects
deal_list: list = [
    # Deal
    'a_deal',
    'a_deal_meta',
    'deal_passport',
    'a_deal_finance',
    'a_deal_check',
    'a_deal_test',
    # Contractor
    'a_contractor',
    'a_contractor_meta',
    'a_deal_contractor',
    'a_deal_contractor_meta',
    # Object
    'a_object',
    'a_object_meta',
    'a_deal_object']

## Main Variables

In [20]:
# Address and name of your notebook
venv_path = "/home/bigdata/a.umarov/jupyter_P24"
# Exploratory Data Analysis(EDA) table (eda_table)
eda_table = real_estate_list[9] # You can generate full-EDA cycle if you want through the all of tables
                                # Just iterate it !
# Exploratory Data Analysis(EDA) obj as PySpark DataFrame
eda_df = None
# Set the date of checking
sample_data_date = '20221027'

## Useful command and modules (can be transform into the 'importlib'- smart import logic)

In [21]:
import os
import pandas as pd
# To data analyze need the module
import pyspark.pandas as ps
os.environ["PYSPARK_PYTHON"] = f"{venv_path}/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = f"{venv_path}/bin/python3"

import findspark
findspark.init("/opt/dev/spark/spark")
 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, DataFrameNaFunctions, \
                        Window, DataFrame, DataFrameStatFunctions, \
                        Column, Row, GroupedData, functions, types
from pyspark.sql.functions import isnan, when, count, col


conf = SparkConf().setAppName("get_info_from_" + eda_table + "_table")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

W1106 20:34:21.357280 32537 process.cpp:2877] Attempted to spawn already running process version@10.62.4.11:46443
I1106 20:34:21.357878 44667 sched.cpp:241] Version: 1.11.0
I1106 20:34:21.358186 32878 sched.cpp:345] New master detected at master@10.62.4.8:5050
I1106 20:34:21.358253 32878 sched.cpp:365] No credentials provided. Attempting to register without authentication
I1106 20:34:21.360110 32884 sched.cpp:753] Framework registered with 89161c75-8817-4ae4-ab24-cba26241d6f9-53173


## Read the spec-file from Hadoop and show some data sample

In [23]:
eda_df = spark.read.parquet("/prod/source/p24/" + eda_table + "/datekey=" + sample_data_date)
eda_df.show(10, False)

+------+---------+--------+-----------+-----------------+------------+
|id    |object_id|metro_id|description|time_on_transport|time_on_walk|
+------+---------+--------+-----------+-----------------+------------+
|100000|227857   |131     |null       |null             |11          |
|100001|227857   |209     |null       |null             |10          |
|100006|238301   |205     |null       |2                |5           |
|100007|238301   |206     |null       |8                |20          |
|100008|238301   |237     |null       |3                |10          |
|100012|238153   |36      |null       |17               |8           |
|100013|238153   |218     |null       |10               |12          |
|100014|238153   |14      |null       |10               |19          |
|100022|238219   |197     |null       |3                |7           |
|100063|237928   |69      |null       |null             |11          |
+------+---------+--------+-----------+-----------------+------------+
only s

## Show columns' properties

In [24]:
print('Data overview')
eda_df.printSchema()

Data overview
root
 |-- id: string (nullable = true)
 |-- object_id: string (nullable = true)
 |-- metro_id: string (nullable = true)
 |-- description: string (nullable = true)
 |-- time_on_transport: string (nullable = true)
 |-- time_on_walk: string (nullable = true)



## Total rows (count)

In [25]:
eda_df.count()

18845

## Simple func to execute and show the result of the SQL-instruction

In [26]:
def execute(select_instruction: str = "select count(*) from realEstate") -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "select count(*) from realEstate"
    :param :string select_instruction: SQL statement
    :return: result of the SQL-operation as a table
    """
    return spark.sql(select_instruction).show()

## Create the temp-table-view from the DF (called as "eda_sql")

In [27]:
eda_df.createOrReplaceTempView("eda_sql")

## Check the method and temp-table

In [28]:
execute("select count(*) from eda_sql")

+--------+
|count(1)|
+--------+
|   18845|
+--------+



## Detect missing values and abnormal zeroes and print it

###### Check-rule Matrix


|  Name of the rule    |Available                   
|----------------------|-------------------------|
|NULL/NaN/NA           |+                        |
|'Zero'-value          |+                        |
|'Empty-string'        |+                        |

### 1st solution (PySpark & Pandas version)
#### Max perfomance but harder to read & understand

##### Filter and count after - all the values in the  our EDA-DF

In [29]:
# NULL/NaN/NA values condition
eda_df_null_values = eda_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_null_values

,id,object_id,metro_id,description,time_on_transport,time_on_walk
0,0,0,0,18845,9799,7170


In [30]:
# Zero values condition
eda_df_zero_values = eda_df.select([count(when(col(c) == 0, c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_zero_values

,id,object_id,metro_id,description,time_on_transport,time_on_walk
0,0,0,0,0,70,21


In [31]:
# Empty-string values condition
eda_df_empty_values = eda_df.select([count(when(col(c) == '', c)).alias(c) \
                                     for c in eda_df.columns]).toPandas()
eda_df_empty_values

,id,object_id,metro_id,description,time_on_transport,time_on_walk
0,0,0,0,0,0,0


In [32]:
# Unlock the limit to display all the columns
pd.options.display.max_rows = None
# To transpose a dataframe - use .T method
eda_df_pandas = pd.concat([eda_df_null_values, eda_df_zero_values, eda_df_empty_values], axis = 'index').T
# To rename columns to anything else after the transpose
eda_df_pandas.columns = (['NULL','Zero','Empty_string'])
eda_df_pandas

,NULL,Zero,Empty_string
id,0,0,0
object_id,0,0,0
metro_id,0,0,0
description,18845,0,0
time_on_transport,9799,70,0
time_on_walk,7170,21,0


In [33]:
def set_result_to_percent(df: object = None) -> object:
    """
    Method to execute and show the result of your PandasDataFrame
    :default -> None-object
    :param :Pandas DF
    :return: DF after the conversion (absolute -> percent)
    """
    if df is None:
        return "Please incl. at least one Pandas DF as a parameter"
    return round(df * 100 / eda_df.count(), 3)

In [34]:
set_result_to_percent(eda_df_pandas)

,NULL,Zero,Empty_string
id,0.000,0.000,0.0
object_id,0.000,0.000,0.0
metro_id,0.000,0.000,0.0
description,100.000,0.000,0.0
time_on_transport,51.998,0.371,0.0
time_on_walk,38.047,0.111,0.0


### SQL-instructions

In [39]:
#Find dupes or misunderstanding or useless static values 18 845 total
for col in eda_df.columns: 
    execute("select count(distinct("+ col + ")) from eda_sql")
    
        
#Check all the 'time-to-target-attrs' (time_on_walk and time_on_transport)
execute("SELECT time_on_walk\
        FROM eda_sql\
        WHERE time_on_walk IS NOT NULL\
            AND time_on_walk != 0\
        ORDER BY time_on_walk desc")    




+------------------+
|count(DISTINCT id)|
+------------------+
|             18845|
+------------------+

+-------------------------+
|count(DISTINCT object_id)|
+-------------------------+
|                    11472|
+-------------------------+

+------------------------+
|count(DISTINCT metro_id)|
+------------------------+
|                     408|
+------------------------+

+---------------------------+
|count(DISTINCT description)|
+---------------------------+
|                          0|
+---------------------------+

+---------------------------------+
|count(DISTINCT time_on_transport)|
+---------------------------------+
|                               86|
+---------------------------------+

+----------------------------+
|count(DISTINCT time_on_walk)|
+----------------------------+
|                         105|
+----------------------------+

+------------+
|time_on_walk|
+------------+
|          99|
|          96|
|          94|
|          90|
|           9|
|        

## Bonus

### 2nd solution (Cycle and variables)
#### Slower but more simple and has more readability than the previous method 

### Create the list of variables to re-use if needed

In [ ]:
useless_columns = []
null_values = []
zero_values = []
empty_string = []
col_name = []

In [ ]:
for col in eda_df.columns:
    null = round((eda_df.filter(eda_df[col].isNull()).count()/eda_df.count())*100, 3)
    zero = round(eda_df.filter(eda_df[col]==0).count()/eda_df.count()*100, 3)
    empty = round(eda_df.filter(eda_df[col]=='').count()/eda_df.count()*100, 3)
    if null in [0, 100] and zero in [0, 100] and empty in [0, 100]:
        useless_columns.append(col)
        continue
    null_values.append(null)
    zero_values.append(zero)
    empty_string.append(empty)
    col_name.append(col)

In [ ]:
zipped = list(zip(null_values, zero_values, empty_string))
eda_df_pandas2 = pd.DataFrame(zipped, index=col_name, columns=['NULL', 'Zero', 'Empty'])
eda_df_pandas2

### Remove useless columns

In [ ]:
eda_df = eda_df.drop(*useless_columns)

In [ ]:
pd.DataFrame(eda_df.dtypes, columns = ['Column Name','Data type'])

In [40]:
spark.stop()

I1106 20:49:25.296774 32537 sched.cpp:2174] Asked to stop the driver
I1106 20:49:25.296875 32860 sched.cpp:1207] Stopping framework 89161c75-8817-4ae4-ab24-cba26241d6f9-53173
